# Inventory Management
## Question 1

**a)** Explain the cost trade-off that is solved by lot-sizing models. (3 points)

**b)** Consider the general newsvendor problem. How does a change in variance affect the optimal order quantity? (4 points)

The Weighty trash Bag Company has fairly constant demand level of 600 units per year. The accounting department estimates that the fixed cost of placing an order is €8. Holding costs are based on a 20 percent annual interest rate. The supplier charges €0.30 per bag for orders of less than 500 bags and €0.29 per bag for orders of 500 bags or more.

**c)** Determine the optimal order quantity for the company under this price schedule. (5 points)

In [37]:
import math

# Given data
D = 600  # annual demand
S = 8  # fixed ordering cost
i = 0.20  # holding cost rate
price_levels = [(0.30, 0, 499), (0.29, 500, float("inf"))]  # (unit cost, min Q, max Q)

# Store results
results = []

for price, q_min, q_max in price_levels:
    h = i * price
    Q = math.sqrt((2 * D * S) / h)

    # Adjust EOQ to meet min quantity condition for price break
    if Q < q_min:
        Q = q_min
    elif Q > q_max:
        Q = q_max

    # Calculate total annual cost
    purchase_cost = D * price
    ordering_cost = S * (D / Q)
    holding_cost = h * (Q / 2)
    total_cost = purchase_cost + ordering_cost + holding_cost

    results.append({"unit_price": price, "order_quantity": Q, "total_cost": total_cost})

# Find optimal solution
optimal = min(results, key=lambda x: x["total_cost"])

# Display results
for r in results:
    print(
        f"Unit price: €{r['unit_price']:.2f}, Order quantity: {r['order_quantity']:.2f}, Total cost: €{r['total_cost']:.2f}"
    )

print("\n✅ Optimal solution:")
print(
    f"Order {optimal['order_quantity']:.2f} units per order at €{optimal['unit_price']:.2f} per unit."
)

Unit price: €0.30, Order quantity: 400.00, Total cost: €204.00
Unit price: €0.29, Order quantity: 500.00, Total cost: €198.10

✅ Optimal solution:
Order 500.00 units per order at €0.29 per unit.


The Weighty trash Bag Company has fairly constant demand level of 600 units per year.
The accounting department estimates that the fixed cost of placing an order is €8. Holding
costs are based on a 20 percent annual interest rate. The supplier charges €0.30 per bag
for orders of less than 500 bags and €0.29 per bag for orders of 500 bags or more.

**d)** Using the convexity of the cost function in Q, compute the optimal order quantity. (4 points)

In [38]:
import math

# Given data
D = 600
S = 8
i = 0.20
c = 0.30
h = i * c
box_size = 250

# Step 1: Compute EOQ without constraints
Q_eoq = math.sqrt((2 * D * S) / h)

# Step 2: Find multiples of 250 around EOQ
lower_multiple = box_size * math.floor(Q_eoq / box_size)
upper_multiple = box_size * math.ceil(Q_eoq / box_size)


# Function to calculate total cost
def total_cost(Q):
    purchase = D * c
    ordering = S * (D / Q)
    holding = h * (Q / 2)
    return purchase + ordering + holding


# Step 3: Evaluate total cost for both multiples
candidates = [lower_multiple, upper_multiple]
results = []

for Q in candidates:
    cost = total_cost(Q)
    results.append((Q, cost))

# Step 4: Choose optimal order quantity
optimal_order = min(results, key=lambda x: x[1])

print(f"EOQ without constraints: {Q_eoq:.2f} units")
print(f"Candidate multiples of {box_size}: {results}")
print(
    f"\n✅ Optimal order quantity: {optimal_order[0]} units with total cost €{optimal_order[1]:.2f}"
)

EOQ without constraints: 400.00 units
Candidate multiples of 250: [(250, 206.7), (500, 204.6)]

✅ Optimal order quantity: 500 units with total cost €204.60


**e)** Assume that we use the order quantity calculated in part d) and there is a lead time of 2 months. What should be the reorder point to ensure that there are no stockouts? (4 points)

In [39]:
# Given data
D = 600  # annual demand units
lead_time_months = 2

# Calculate monthly demand
monthly_demand = D / 12

# Calculate reorder point
ROP = monthly_demand * lead_time_months

print(f"Reorder Point (ROP) to avoid stockouts: {ROP:.2f} units")

Reorder Point (ROP) to avoid stockouts: 100.00 units


Consider two newsvendors A and B. Their demands are assumed to be normally
distributed. The two newsvendors calculated the respective means as 𝜇A= 80, 𝜇B= 60
and standard deviations 𝜎A = 𝜎B = 20. The demand of newsvendors A and B is negatively
correlated with coefficient -0.5. The sales price is 100 €, the procurement price is 30 €
and leftover inventory is discarded at no cost.

**f)** Determine the individual optimal order quantities and the related profits if newsvendors procure independently. (6 points)

In [40]:
import numpy as np
from scipy.stats import norm, poisson, gamma
from scipy.integrate import quad


def newsvendor(label, distr, params, p, c, g=0):
    """
    Generalized newsvendor solution for different demand distributions.

    distr: string, one of ['normal', 'poisson', 'gamma']
    params: dict, parameters of the distribution
        - normal: {'mu':..., 'sigma':...}
        - poisson: {'lambda': ...}
        - gamma: {'mu':..., 'sigma':...}
    p, c, g: prices (selling, cost, salvage)

    Returns dict with order quantity, expected sales, profit, etc.
    """

    CR = (p - c) / (p - g)

    if distr == "normal":
        mu = params["mu"]
        sigma = params["sigma"]
        z = norm.ppf(CR)
        Q = mu + z * sigma

        phi_z = norm.pdf(z)
        Phi_z = norm.cdf(z)
        G_z = phi_z - z * (1 - Phi_z)
        expected_lost_sales = sigma * G_z
        expected_sales = mu - expected_lost_sales
        expected_leftover_inventory = Q - expected_sales
        expected_profit = -c * Q + p * expected_sales + g * expected_leftover_inventory
        non_stockout_prob = Phi_z
        fill_rate = expected_sales / mu

        return {
            "Label": label,
            "Distribution": "Normal",
            "Critical ratio (CR)": CR,
            "z": z,
            "phi(z)": phi_z,
            "Phi(z)": Phi_z,
            "G(z)": G_z,
            "Order Quantity": Q,
            "Expected Lost Sales": expected_lost_sales,
            "Expected Sales": expected_sales,
            "Expected Leftover Inventory": expected_leftover_inventory,
            "Expected Profit": expected_profit,
            "Non-stockout Probability (alpha)": non_stockout_prob,
            "Fill Rate (beta)": fill_rate,
        }

    elif distr == "poisson":
        lam = params["lambda"]
        Q = 0
        while poisson.cdf(Q, lam) < CR:
            Q += 1

        # Expected lost sales = sum_{k=Q+1}^∞ (k - Q)*P(D=k)
        # We'll approximate with upper bound Q+1000
        max_k = Q + 1000
        expected_lost_sales = sum(
            (k - Q) * poisson.pmf(k, lam) for k in range(Q + 1, max_k + 1)
        )
        expected_sales = lam - expected_lost_sales
        expected_leftover_inventory = Q - expected_sales
        expected_profit = -c * Q + p * expected_sales + g * expected_leftover_inventory
        non_stockout_prob = poisson.cdf(Q, lam)
        fill_rate = expected_sales / lam

        return {
            "Label": label,
            "Distribution": "Poisson",
            "Critical ratio (CR)": CR,
            "Order Quantity": Q,
            "Expected Lost Sales": expected_lost_sales,
            "Expected Sales": expected_sales,
            "Expected Leftover Inventory": expected_leftover_inventory,
            "Expected Profit": expected_profit,
            "Non-stockout Probability (alpha)": non_stockout_prob,
            "Fill Rate (beta)": fill_rate,
        }

    elif distr == "gamma":
        mu = params["mu"]
        sigma = params["sigma"]

        alpha = (mu**2) / (sigma**2)
        theta = (sigma**2) / mu

        Q = gamma.ppf(CR, a=alpha, scale=theta)

        def integrand(x):
            return x * gamma.pdf(x, a=alpha, scale=theta)

        integral_val, _ = quad(integrand, 0, Q)
        expected_sales = integral_val + Q * (1 - gamma.cdf(Q, a=alpha, scale=theta))
        expected_leftover_inventory = Q - expected_sales
        expected_lost_sales = mu - integral_val
        expected_profit = -c * Q + p * expected_sales + g * expected_leftover_inventory
        non_stockout_prob = gamma.cdf(Q, a=alpha, scale=theta)
        fill_rate = expected_sales / mu

        return {
            "Label": label,
            "Distribution": "Gamma",
            "Critical ratio (CR)": CR,
            "Shape (alpha)": alpha,
            "Scale (theta)": theta,
            "Order Quantity": Q,
            "Expected Lost Sales": expected_lost_sales,
            "Expected Sales": expected_sales,
            "Expected Leftover Inventory": expected_leftover_inventory,
            "Expected Profit": expected_profit,
            "Non-stockout Probability (alpha)": non_stockout_prob,
            "Fill Rate (beta)": fill_rate,
        }

    else:
        raise ValueError("Unsupported distribution")


# === Example usage ===

print("Normal distribution example:")
res_norm = newsvendor("ExA", "normal", {"mu": 80, "sigma": 20}, p=100, c=30, g=0)
for k, v in res_norm.items():
    print(f"{k:50s}: {v:.2f}" if isinstance(v, float) else f"{k:30s}: {v}")

print("\nPoisson distribution example:")
res_pois = newsvendor("ExB", "poisson", {"lambda": 3}, p=10, c=4, g=0)
for k, v in res_pois.items():
    print(f"{k:50s}: {v:.2f}" if isinstance(v, float) else f"{k:30s}: {v}")

print("\nGamma distribution example:")
res_gamma = newsvendor("ExC", "gamma", {"mu": 3, "sigma": 1.732}, p=10, c=4, g=0)
for k, v in res_gamma.items():
    print(f"{k:50s}: {v:.2f}" if isinstance(v, float) else f"{k:30s}: {v}")

Normal distribution example:
Label                         : ExA
Distribution                  : Normal
Critical ratio (CR)                               : 0.70
z                                                 : 0.52
phi(z)                                            : 0.35
Phi(z)                                            : 0.70
G(z)                                              : 0.19
Order Quantity                                    : 90.49
Expected Lost Sales                               : 3.81
Expected Sales                                    : 76.19
Expected Leftover Inventory                       : 14.30
Expected Profit                                   : 4904.61
Non-stockout Probability (alpha)                  : 0.70
Fill Rate (beta)                                  : 0.95

Poisson distribution example:
Label                         : ExB
Distribution                  : Poisson
Critical ratio (CR)                               : 0.60
Order Quantity                : 3
Expected

In [41]:
# Given data
(
    mu_A,
    sigma_A,
) = (
    80,
    20,
)
mu_B, sigma_B = 60, 20
p, c, g = 100, 30, 0

# Calculate for A and B
results_A = newsvendor(
    "Newsvendor A", "normal", {"mu": mu_A, "sigma": sigma_A}, p=100, c=30, g=0
)
for k, v in results_A.items():
    print(f"{k:50s}: {v:.2f}" if isinstance(v, float) else f"{k:30s}: {v}")

print("\n")

results_B = newsvendor(
    "Newsvendor B", "normal", {"mu": mu_B, "sigma": sigma_B}, p=100, c=30, g=0
)
for k, v in results_B.items():
    print(f"{k:50s}: {v:.2f}" if isinstance(v, float) else f"{k:30s}: {v}")

Label                         : Newsvendor A
Distribution                  : Normal
Critical ratio (CR)                               : 0.70
z                                                 : 0.52
phi(z)                                            : 0.35
Phi(z)                                            : 0.70
G(z)                                              : 0.19
Order Quantity                                    : 90.49
Expected Lost Sales                               : 3.81
Expected Sales                                    : 76.19
Expected Leftover Inventory                       : 14.30
Expected Profit                                   : 4904.61
Non-stockout Probability (alpha)                  : 0.70
Fill Rate (beta)                                  : 0.95


Label                         : Newsvendor B
Distribution                  : Normal
Critical ratio (CR)                               : 0.70
z                                                 : 0.52
phi(z)                    

**g)** If newsvendors A and B cooperate and give joint orders, what should be the joint
order quantity? Without calculating associated (total) profit, explain how it would be
affected and why? Name this effect and give a short explanation. (4 points)

In [42]:
import numpy as np
from scipy.stats import norm

# Given data
mu_A, sigma_A = 80, 20
mu_B, sigma_B = 60, 20
rho = -0.5  # correlation coefficient
p, c, g = 100, 30, 0  # selling price, cost, salvage

# Critical ratio and z-score
CR = (p - c) / (p - g)
z_CR = norm.ppf(CR)

# Individual order quantities
Q_A = mu_A + z_CR * sigma_A
Q_B = mu_B + z_CR * sigma_B

# Joint demand mean
mu_joint = mu_A + mu_B

# Joint demand std dev with correlation
sigma_joint = np.sqrt(sigma_A**2 + sigma_B**2 + 2 * rho * sigma_A * sigma_B)

# Joint order quantity
Q_joint = mu_joint + z_CR * sigma_joint

print("=== Individual Newsvendors ===")
print(f"Newsvendor A Order Quantity: {Q_A:.2f}")
print(f"Newsvendor B Order Quantity: {Q_B:.2f}")
print(f"Sum of Individual Order Quantities: {Q_A + Q_B:.2f}")

print("\n=== Joint (Pooled) Newsvendor with Correlation ===")
print(f"Joint Demand Mean (mu_joint): {mu_joint:.2f}")
print(f"Joint Demand Std Dev (sigma_joint): {sigma_joint:.2f}")
print(f"Joint Order Quantity: {Q_joint:.2f}")

print("\n=== Risk Pooling Effect Explanation ===")
print("Negative correlation reduces joint demand variability more than independence.")
print(f"Coefficient of Variation A: {sigma_A/mu_A:.4f}")
print(f"Coefficient of Variation B: {sigma_B/mu_B:.4f}")
print(f"Coefficient of Variation Joint: {sigma_joint/mu_joint:.4f}")
print(
    "\nBecause of the negative correlation, the joint demand uncertainty is even lower,"
)
print("allowing for a smaller safety stock and lower total inventory.")
print("This phenomenon is part of the 'risk pooling effect'.")

=== Individual Newsvendors ===
Newsvendor A Order Quantity: 90.49
Newsvendor B Order Quantity: 70.49
Sum of Individual Order Quantities: 160.98

=== Joint (Pooled) Newsvendor with Correlation ===
Joint Demand Mean (mu_joint): 140.00
Joint Demand Std Dev (sigma_joint): 20.00
Joint Order Quantity: 150.49

=== Risk Pooling Effect Explanation ===
Negative correlation reduces joint demand variability more than independence.
Coefficient of Variation A: 0.2500
Coefficient of Variation B: 0.3333
Coefficient of Variation Joint: 0.1429

Because of the negative correlation, the joint demand uncertainty is even lower,
allowing for a smaller safety stock and lower total inventory.
This phenomenon is part of the 'risk pooling effect'.


# Question 2

**a)** Explain (without formulas) the differences between (R, S), (s, Q), and (s, S) inventory control rules. Discuss the parameters s, Q and S and how these parameters
should be set? (6 points)

Canadian Wheel Ltd. continuously reviews its inventory and establishes safety stocks
to satisfy a certain service level. For a basic item whose demand is normally distributed,
an order quantity of 200 units is used. Annual demand for the item is 1000 units and its
weekly standard deviation is 50. The supplier of the item ensures a constant Iead time
of 4 weeks to Canadian Wheel. The current purchase price of the item from the supplier
is €0.80/unit. Receiving and handling costs add €0.20/unit. Annual carrying charge is 20
percent and penalty cost of p=€0.50 per unit short applies.

The supplier offers to reduce the Iead time to a new constant level of one week, but in
so doing she will increase the selling price to Canadian by €0.05/unit. Canadian
management is faced with the decision of whether or not to accept the supplier’s offer.

**b)** Discuss the economic tradeoff between current and new supplier lead time/price. (6 points)

**c)** Determine the expected number of stock-outs per replenishment cycle and calculate
the safety stocks required in the present and offered contract if the company aims a
cycle fill-rate of 0.94. (5 points)

In [43]:
from scipy.stats import norm
import math

# Inputs
sigma_d = 50
L_old = 4
L_new = 1
cycle_fill_rate = 0.94
Q = 200

# Z-score for fill rate β = 0.94
z_beta = norm.ppf(cycle_fill_rate)

# Safety stock (constant lead time)
ss_old = z_beta * sigma_d * math.sqrt(L_old)
ss_new = z_beta * sigma_d * math.sqrt(L_new)

# Expected stock-outs per cycle (Q * (1 - fill rate))
stockouts_per_cycle = Q * (1 - cycle_fill_rate)

# Output
print("=== Part (c): Fill Rate = 0.94 ===")
print(f"Z-score (β = 0.94): {z_beta:.4f}")
print(f"Safety Stock (current contract, L=4): {ss_old:.2f} units")
print(f"Safety Stock (new contract, L=1):    {ss_new:.2f} units")
print(f"Expected Stock-Outs per Replenishment Cycle: {stockouts_per_cycle:.2f} units")

=== Part (c): Fill Rate = 0.94 ===
Z-score (β = 0.94): 1.5548
Safety Stock (current contract, L=4): 155.48 units
Safety Stock (new contract, L=1):    77.74 units
Expected Stock-Outs per Replenishment Cycle: 12.00 units


**d)** Do the same as c) for α-service level of 0.90. (4 points)

In [44]:
# Inputs
sigma_d = 50
L_old = 4
L_new = 1
alpha_service_level = 0.90

# Z-score for α-service level = 0.90
z_alpha = norm.ppf(alpha_service_level)

# Safety stock (constant lead time)
ss_old_alpha = z_alpha * sigma_d * math.sqrt(L_old)
ss_new_alpha = z_alpha * sigma_d * math.sqrt(L_new)

# Expected stockouts per cycle
Q = 200  # order quantity remains the same
expected_stockout_alpha = (1 - alpha_service_level) * Q

print("=== Part (d): α-Service Level = 0.90 ===")
print(f"Z-score (α = 0.90): {z_alpha:.4f}")
print(f"Safety Stock (current contract, L=4): {ss_old_alpha:.2f} units")
print(f"Safety Stock (new contract, L=1):    {ss_new_alpha:.2f} units")
print(f"Expected Stockouts per Cycle:        {expected_stockout_alpha:.2f} units")

=== Part (d): α-Service Level = 0.90 ===
Z-score (α = 0.90): 1.2816
Safety Stock (current contract, L=4): 128.16 units
Safety Stock (new contract, L=1):    64.08 units
Expected Stockouts per Cycle:        20.00 units


**e)** What should be the decision of whether or not to accept the supplier’s offer
considering the annual cost of each contract for the case that the company aims a
cycle fill-rate of 0.94? (9 points)

In [45]:
from scipy.stats import norm
import math

# Constants
D = 1000
Q = 200
sigma_week = 50
weeks_per_year = 52
p_old = 0.80
p_new = 0.85
handling_cost = 0.20
carrying_rate = 0.20
penalty_cost = 0.50
service_level = 0.94

# Lead times (weeks)
L_old = 4
L_new = 1

# Std dev demand over lead time
sigma_L_old = sigma_week * math.sqrt(L_old)
sigma_L_new = sigma_week * math.sqrt(L_new)

# Z value for cycle fill rate = 0.94
z = norm.ppf(service_level)

# Safety stock calculation
ss_old = z * sigma_L_old
ss_new = z * sigma_L_new

# Number of cycles per year
cycles = D / Q

# Expected stockouts per cycle from formula: Expected lost sales per cycle
# ≈ (1 - cycle fill rate) * Q (approximation)
expected_stockouts = (1 - service_level) * Q


# Calculate annual costs
def annual_cost(purchase_price):
    purchase = D * purchase_price
    handling = D * handling_cost
    holding = (
        carrying_rate * purchase_price * (Q / 2 + ss_old)
    )  # We'll calculate holding cost twice, once for old and once for new, so parameterize ss.
    penalty = expected_stockouts * penalty_cost * cycles
    return purchase + handling + holding + penalty


# Old contract cost
holding_old = carrying_rate * p_old * (Q / 2 + ss_old)
annual_cost_old = (
    D * p_old
    + D * handling_cost
    + holding_old
    + expected_stockouts * penalty_cost * cycles
)

# New contract cost (recalculate holding cost with new ss and price)
holding_new = carrying_rate * p_new * (Q / 2 + ss_new)
annual_cost_new = (
    D * p_new
    + D * handling_cost
    + holding_new
    + expected_stockouts * penalty_cost * cycles
)

print(f"Safety stock (old): {ss_old:.2f} units")
print(f"Safety stock (new): {ss_new:.2f} units")
print(f"Annual cost (old contract): €{annual_cost_old:.2f}")
print(f"Annual cost (new contract): €{annual_cost_new:.2f}")

if annual_cost_new < annual_cost_old:
    print("Decision: Accept the new supplier offer.")
else:
    print("Decision: Keep the current supplier contract.")

Safety stock (old): 155.48 units
Safety stock (new): 77.74 units
Annual cost (old contract): €1070.88
Annual cost (new contract): €1110.22
Decision: Keep the current supplier contract.


# Question 3

**a)** Explain dedicated capacity, average utilization, and common cycle approaches. (8 points)

Consider two products with constant demand rate of d1 = 300 and d2 = 350 units per
period which are stored in a warehouse with a total capacity of 500 units. The
products require a1 = 3 and a2 = 1 units of warehouse space. The ordering costs are
A1=150€ and A2=111€. Additionally, the products cause holding costs of h1=1.5€ and
h2 = 2€ per period

**b)** Determine, using the strategy of dedicated space, the optimal order quantities.
(Hint: Consider only integer values for the Lagrange multiplier)  (7 points)

In [46]:
import numpy as np

d1, d2 = 300, 350
a1, a2 = 3, 1
A1, A2 = 150, 111
h1, h2 = 1.5, 2
W = 500

def compute_Q(lambda_):
    Q1 = np.sqrt((2 * A1 * d1) / (h1 + 2 * lambda_ * a1))
    Q2 = np.sqrt((2 * A2 * d2) / (h2 + 2 * lambda_ * a2))
    return Q1, Q2


def total_space(Q1, Q2):
    return a1 * Q1 + a2 * Q2


def total_cost(Q1, Q2):
    cost1 = (A1 * d1) / Q1 + (h1 * Q1) / 2
    cost2 = (A2 * d2) / Q2 + (h2 * Q2) / 2
    return cost1 + cost2


# Search for optimal lambda (integer values 0 to 10)
results = []
for lam in range(11):
    Q1, Q2 = compute_Q(lam)
    space_used = total_space(Q1, Q2)
    feasible = space_used <= W
    cost = total_cost(Q1, Q2)
    results.append(
        {
            "lambda": lam,
            "Q1": Q1,
            "Q2": Q2,
            "space_used": space_used,
            "feasible": feasible,
            "total_cost": cost,
        }
    )

# Filter feasible solutions and pick the one with minimal total cost
feasible_results = [r for r in results if r["feasible"]]
optimal = (
    min(feasible_results, key=lambda x: x["total_cost"]) if feasible_results else None
)

# Print results
print(
    f"{'lambda':>6} | {'Q1':>8} | {'Q2':>8} | {'Space Used':>11} | {'Feasible':>8} | {'Total Cost':>11}"
)
print("-" * 85)
for r in results:
    print(
        f"{r['lambda']:6d} | {r['Q1']:8.2f} | {r['Q2']:8.2f} | {r['space_used']:11.2f} | {str(r['feasible']):>8} | {r['total_cost']:11.2f}"
    )

if optimal:
    print("\nOptimal solution:")
    print(f"lambda = {optimal['lambda']}")
    print(f"Q1 = {optimal['Q1']:.2f}")
    print(f"Q2 = {optimal['Q2']:.2f}")
    print(f"Total space used = {optimal['space_used']:.2f}")
    print(f"Total cost = {optimal['total_cost']:.2f}")
else:
    print("No feasible solution found within lambda range.")

lambda |       Q1 |       Q2 |  Space Used | Feasible |  Total Cost
-------------------------------------------------------------------------------------
     0 |   244.95 |   197.10 |      931.95 |    False |      761.63
     1 |   109.54 |   139.37 |      468.01 |     True |      911.07
     2 |    81.65 |   113.80 |      358.75 |     True |     1067.56
     3 |    67.94 |    98.55 |      302.36 |     True |     1206.09
     4 |    59.41 |    88.15 |      266.37 |     True |     1330.91
     5 |    53.45 |    80.47 |      240.82 |     True |     1445.23
     6 |    48.99 |    74.50 |      221.47 |     True |     1551.29
     7 |    45.49 |    69.69 |      206.14 |     True |     1650.61
     8 |    42.64 |    65.70 |      193.62 |     True |     1744.34
     9 |    40.27 |    62.33 |      183.14 |     True |     1833.30
    10 |    38.25 |    59.43 |      174.19 |     True |     1918.17

Optimal solution:
lambda = 1
Q1 = 109.54
Q2 = 139.37
Total space used = 468.01
Total cost = 911.0

**c)** How much should be paid for increasing warehouse space to 1000 units? (5 points)

In [47]:
import numpy as np

# Parameters for N products
d = np.array([300, 350])       # Demand
a = np.array([3, 1])           # Workload per unit
A = np.array([150, 111])       # Ordering cost
h = np.array([1.5, 2.0])       # Holding cost per unit per period

def compute_Q(lambda_):
    """Compute EOQ Q_i for all products with given lambda."""
    Q = np.sqrt((2 * A * d) / (h + 2 * lambda_ * a))
    return Q

def total_space(Q):
    """Compute total workload space used by Q."""
    return np.sum(a * Q)

def total_cost(Q):
    """Compute total cost for given Q."""
    ordering_costs = np.sum((A * d) / Q)
    holding_costs = np.sum((h * Q) / 2)
    return ordering_costs + holding_costs

def find_optimal_cost(W, lambda_range=range(50)):
    """Search for optimal cost under workload constraint W."""
    feasible_costs = []
    for lam in lambda_range:
        Q = compute_Q(lam)
        space = total_space(Q)
        if space <= W:
            cost = total_cost(Q)
            feasible_costs.append((lam, cost, Q, space))
    if feasible_costs:
        lam_star, min_cost, Q_star, used_space = min(feasible_costs, key=lambda x: x[1])
        return min_cost, lam_star, Q_star, used_space
    else:
        return float("inf"), None, None, None

# Compare for different warehouse capacities
cost_500, lambda_500, Q_500, _ = find_optimal_cost(500)
cost_1000, lambda_1000, Q_1000, _ = find_optimal_cost(1000)

value_of_expansion = cost_500 - cost_1000

print(f"Kosten bei 500 Lagerkapazität:  €{cost_500:.2f} (λ = {lambda_500})")
print(f"Kosten bei 1000 Lagerkapazität: €{cost_1000:.2f} (λ = {lambda_1000})")
print(f"\nMax. Zahlungsbereitschaft für Erweiterung: €{value_of_expansion:.2f}")


Kosten bei 500 Lagerkapazität:  €911.07 (λ = 1)
Kosten bei 1000 Lagerkapazität: €761.63 (λ = 0)

Max. Zahlungsbereitschaft für Erweiterung: €149.44


**d)** Determine, using the Common-cycle method, the optimal replenishment cycle for
all products. (5 points)

In [48]:
import math

# Input Data
d = [300, 350]           # Demand per period
A = [150, 111]           # Ordering cost
h = [1.5, 2.0]           # Holding cost per unit per period
a = [3, 1]               # Workload per unit
W = 500                  # Max workload per period

# Step 1: Unconstrained Case
sum_A = sum(A)
sum_hd = sum(h[i] * d[i] for i in range(len(d)))

T_unc = math.sqrt(2 * sum_A / sum_hd)
print(f"1) Unconstrained cycle time T_unc: {T_unc:.3f}")

# Step 2: Constrained Case
numerator = sum(a[i] * d[i] for i in range(len(d)))
print(f"Numerator: {numerator}")

denominator = 0
for i in range(len(d)):
    for j in range(i + 1):  # includes j = i
        term = a[i] * a[j] * d[i] * d[j]
        denominator += term
        print(f"a[{i}]*a[{j}]*d[{i}]*d[{j}] = {term}")


print(f"Denominator: {denominator}")

T_cons = W * (numerator / denominator)
print(f"2) Constrained cycle time T_cons: {T_cons:.3f}")

# Step 3: Take the minimum
T_star = min(T_unc, T_cons)
print(f"3) Optimal common replenishment cycle T*: {T_star:.3f}")


1) Unconstrained cycle time T_unc: 0.674
Numerator: 1250
a[0]*a[0]*d[0]*d[0] = 810000
a[1]*a[0]*d[1]*d[0] = 315000
a[1]*a[1]*d[1]*d[1] = 122500
Denominator: 1247500
2) Constrained cycle time T_cons: 0.501
3) Optimal common replenishment cycle T*: 0.501


**e)** Use the results in d) to determine how many units of product 1 are in stock when
you replenish product 2? (5 points)
 (5 points)

In [49]:

# Step 4: Calculate Q_i*
Q = [d[i] * T_star for i in range(len(d))]
for i, q in enumerate(Q):
    print(f"Q*[{i+1}] (order quantity for product {i+1}): {q:.2f}")

# Step 5: Calculate replenishment times t_i
cumulative = 0
replenishment_times = []
for i in range(len(d)):
    cumulative += a[i] * d[i]
    t_i = (cumulative / numerator) * T_star
    replenishment_times.append(t_i)
    print(f"t[{i+1}] (replenishment time for product {i+1}): {t_i:.3f}")

# Step 6: Inventory of product 1 when product 2 is replenished
t1 = replenishment_times[0]
t2 = replenishment_times[1]
Q1 = Q[0]
inventory_product1_at_t2 = Q1 * (1 - (t2 - t1) / T_star)
print(f"4) Inventory of product 1 when product 2 is replenished: {inventory_product1_at_t2:.2f} units")


Q*[1] (order quantity for product 1): 150.30
Q*[2] (order quantity for product 2): 175.35
t[1] (replenishment time for product 1): 0.361
t[2] (replenishment time for product 2): 0.501
4) Inventory of product 1 when product 2 is replenished: 108.22 units


# Question 4

A company has collected the following sales data over the last ten months for product A and B.

| t        | 1  | 2   | 3  | 4  | 5  | 6  | 7   | 8  | 9  | 10 |
|----------|----|-----|----|----|----|----|-----|----|----|----|
| Product A| 4  | 8   | 0  | 3  | 0  | 0  | 0   | 0  | 4  | 0  |
| Product B| 99 | 155 | 75 | 81 | 88 | 150| 101 | 74 | 65 | 48 |


**a)** Classify products A and B using XYZ analysis. (4 points)

In [50]:
import numpy as np

# Sales data for Product A and B over 10 months
sales_A = np.array([4, 8, 0, 3, 0, 0, 0, 0, 4, 0])
sales_B = np.array([99, 155, 75, 81, 88, 150, 101, 74, 65, 48])


def xyz_classification(sales):
    total_sales = np.sum(sales)
    if total_sales == 0:
        return "Z"

    # Calculate coefficient of variation (CV)
    mean_sales = np.mean(sales)
    std_sales = np.std(sales)
    cv = std_sales / mean_sales if mean_sales != 0 else float("inf")

    # Classify based on CV thresholds
    if cv <= 0.5:
        return "X"  # Very stable demand
    elif cv < 1.25:
        return "Y"  # Moderate variability
    else:
        return "Z"  # High variability, more periods without demand than with demand


# Perform XYZ classification for both products
classification_A = xyz_classification(sales_A)
classification_B = xyz_classification(sales_B)

print(f"Product A is classified as: {classification_A}")
print(f"Product B is classified as: {classification_B}")

Product A is classified as: Z
Product B is classified as: X


**b)** Explain the meaning of R² in regression. (4 points)

**c)** Consider the standard newsvendor model. How does the quality of your forecast influence your order decision? (4 points)

Consider a two-stage serial inventory system where the retailer orders from the warehouse. Both parties in the supply chain use an (R, S) inventory control rule. Review period of the retailer is 1 week, and it is immediately supplied by the warehouse with the ordered amount. On the other hand, review period of the warehouse is 0 and it is supplied by an outside supplier after a lead time of 1 week. Demand is normally distributed with unknown level and known standard deviation of 15. The initial guess for the demand level is 50. Both parties are supposed to satisfy an α-service level of 95%.

In the following table weekly demand observations of the retailer are given.

| Week   | 1  | 2  | 3  | 4  | 5  |
|--------|----|----|----|----|----|
| Demand | 56 | 40 | 61 | 42 | 51 |

**d)** Perform an ex-post forecast for the retailer using exponential smoothing with alpha=0.1 and determine the order-up-to level that would have been optimal given the forecast. Calculate the retailer’s order quantities. (4 points)


In [57]:
import numpy as np
from scipy.stats import norm

demand = np.array([56, 40, 61, 42, 51])
alpha = 0.1
initial_forecast = 50
std_dev = 15
z = norm.ppf(0.95)  # 1.645

# Step 1: Forecast demand with exponential smoothing
mu = [initial_forecast]
for t in range(len(demand)):
    mu.append(alpha * demand[t] + (1 - alpha) * mu[-1])
mu = np.array(mu[1:])

# Step 2: Calculate order-up-to levels (S)
S = mu + z * std_dev

# Step 3: Round order-up-to levels up (ceiling)
S_int = np.ceil(S)

# Step 4: Initialize previous S_int to first rounded S (S_0)
S_prev = 75  # initial SR_integer

orders = np.zeros(len(demand), dtype=int)

# Step 5: Calculate orders recursively with corrected formula
for t in range(len(demand)):
    orders[t] = demand[t] - S_prev + S_int[t]
    S_prev = S_int[t]

print("Forecasts (mu):", np.round(mu, 2))
print("Order-up-to levels (S):", np.round(S, 2))
print("Rounded order-up-to levels (S_int):", S_int)
print("Orders (q):", orders)


Forecasts (mu): [50.6  49.54 50.69 49.82 49.94]
Order-up-to levels (S): [75.27 74.21 75.36 74.49 74.61]
Rounded order-up-to levels (S_int): [76. 75. 76. 75. 75.]
Orders (q): [57 39 62 41 51]


**e)** Perform an ex-post forecast for the warehouse using exponential smoothing with alpha=0.1 and determine the order-up-to level that would have been optimal given the forecast. Calculate the warehouse’s order quantities. (4 points)


In [59]:
import numpy as np
from scipy.stats import norm

# Given retailer orders from previous step (qR)
retailer_orders = np.array([57, 39, 62, 41, 51])

alpha = 0.1
initial_forecast = 50
std_dev = 15
z = norm.ppf(0.95)  # 1.645

# Step 1: Forecast wholesaler demand with exponential smoothing
mu_wh = [initial_forecast]
for t in range(len(retailer_orders)):
    mu_wh.append(alpha * retailer_orders[t] + (1 - alpha) * mu_wh[-1])
mu_wh = np.array(mu_wh[1:])

# Step 2: Calculate wholesaler order-up-to levels (SW)
SW = mu_wh + z * std_dev

# Step 3: Round wholesaler order-up-to levels
SW_int = np.ceil(SW)

# Step 4: Initialize previous SW_int
SW_prev = 75

orders_wh = np.zeros(len(retailer_orders), dtype=int)

for t in range(len(retailer_orders)):
    orders_wh[t] = retailer_orders[t] - SW_prev + SW_int[t]
    SW_prev = SW_int[t]

print("Wholesaler forecasts (mu):", np.round(mu_wh, 2))
print("Wholesaler order-up-to levels (SW):", np.round(SW, 2))
print("Wholesaler rounded order-up-to levels (SW_int):", SW_int)
print("Wholesaler orders (qW):", orders_wh)


Wholesaler forecasts (mu): [50.7  49.53 50.78 49.8  49.92]
Wholesaler order-up-to levels (SW): [75.37 74.2  75.45 74.47 74.59]
Wholesaler rounded order-up-to levels (SW_int): [76. 75. 76. 75. 75.]
Wholesaler orders (qW): [58 38 63 40 51]


**f)** Calculate the variance of demand and orders of retailer and the warehouse. (3 points)


In [60]:
import numpy as np

# Data
retailer_demand = np.array([56, 40, 61, 42, 51])
retailer_orders = np.array([57, 39, 62, 41, 51])
wholesaler_orders = np.array([58, 38, 63, 40, 51])

# Function to compute stats
def compute_stats(data, label):
    mean = np.mean(data)
    std_dev = np.std(data, ddof=1)  # Sample standard deviation
    variance = np.var(data, ddof=1)
    
    print(f"{label}")
    print(f"  Mean     : {mean:.2f}")
    print(f"  Std Dev  : {std_dev:.2f}")
    print(f"  Variance : {variance:.2f}")
    print("-" * 30)

# Compute and print stats
compute_stats(retailer_demand, "Retailer Demand")
compute_stats(retailer_orders, "Retailer Orders (qR)")
compute_stats(wholesaler_orders, "Wholesaler Orders (qW)")


Retailer Demand
  Mean     : 50.00
  Std Dev  : 8.97
  Variance : 80.50
------------------------------
Retailer Orders (qR)
  Mean     : 50.00
  Std Dev  : 9.95
  Variance : 99.00
------------------------------
Wholesaler Orders (qW)
  Mean     : 50.00
  Std Dev  : 10.93
  Variance : 119.50
------------------------------


**g)** Which effect can be observed from the results obtained part f)? Name and describe the effect briefly. What are possible strategies to mitigate the effect? (7 points)